In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
    ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam, RMSprop

from utils import get_imagens_erro, report, plot_errors
import time

MODEL_DIR = '../modelschest'
LOG_DIR = '../logs/02d/chestXRay'
SIZE = (224, 224)
caminho_bases = os.path.join('..', 'bases', 'chest_xray')
caminho_train = os.path.join(caminho_bases, 'train')
caminho_test = os.path.join(caminho_bases, 'test')

# Callbacks

In [2]:
tensorboard_logs = TensorBoard(log_dir=LOG_DIR, histogram_freq=1,
                               write_graph=False, write_images=False,
                               update_freq='epoch')
mcp_save = ModelCheckpoint(os.path.join(MODEL_DIR, 
                                        'Transfermodelweights02d.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=4,
                              verbose=1, min_delta=1e-2, mode='min')


# Load features to np

In [2]:
FEATURES_DIR = os.path.join('..', 'bases', 'chestXRay_02cfeatures')
with open(os.path.join(FEATURES_DIR, 'features_train.np'), 'rb') as features_in:
    features_train = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_train.np'), 'rb') as y_in:
    y_train = np.load(y_in)
with open(os.path.join(FEATURES_DIR, 'features_test.np'), 'rb') as features_in:
    features_test = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_test.np'), 'rb') as y_in:
    y_test = np.load(y_in)

# Model

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256,
                        activation='relu',
                        kernel_regularizer=tf.keras.regularizers.l1_l2(l2=0.01, l1=0.01),
                        kernel_initializer='glorot_normal',
                        input_shape=(1024,)),
  tf.keras.layers.Dropout(0.33),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [4]:
model.compile(optimizer=Adam(lr=0.0004), loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


# Training and validation

In [8]:
epochs = 20

def get_epoch(history, number):
    acc = history['acc'][number]
    val_acc = history['val_acc'][number]
    return 'Number: %s - acc: %s  - val_acc: %s' % (number, acc, val_acc)
    

def resume_epochs(history, batch_size):
    print('Optimizer: %s lr: %s Batch Size: %s' % (optimizer_function.__name__, lr, batch_size))
    print( [get_epoch(history.history, 0), 
            get_epoch(history.history, len(history.history['acc']) // 2),
            get_epoch(history.history, len(history.history['acc']) - 1)
           ])

best_val_acc = 0
best_params = None
all_historys = []
for trial in range(3):
    for lr in [0.01, 0.001, 0.0004, 0.0001, 0.00004, 0.00002, 0.00001]:
        for optimizer_function in [RMSprop, Adam]:
            for batch_size in [64, 128, 256]:
                optimizer=optimizer_function(lr=lr)
                model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
                history = model.fit(x=features_train,
                                    y=y_train,
                                    batch_size=batch_size,
                                    epochs=epochs,
                                    verbose=0,
                                    validation_data=(features_test, y_test),
                                   )
                resume_epochs(history, batch_size)
                mean_val_acc = sum(history.history['val_acc']) / epochs
                this_history = {'trial': trial,
                                'lr': lr,
                                'optimizer': optimizer_function.__name__,
                                'batch_size': batch_size,
                                'history': history.history,
                                'mean_val_acc': mean_val_acc}
                all_historys.append(this_history)
                for ind, val_acc in enumerate(history.history['val_acc'], 1):
                    if val_acc > best_val_acc:
                        best_val_acc = val_acc
                        best_params = {**this_history,
                                       'epochs': ind,
                                       'val_acc': val_acc}



Optimizer: RMSprop lr: 0.01 Batch Size: 64
['Number: 0 - acc: 0.66184413  - val_acc: 0.61513156', 'Number: 2 - acc: 0.7008102  - val_acc: 0.61513156', 'Number: 3 - acc: 0.705054  - val_acc: 0.61513156']
Optimizer: RMSprop lr: 0.01 Batch Size: 128
['Number: 0 - acc: 0.66358024  - val_acc: 0.59539473', 'Number: 2 - acc: 0.7175926  - val_acc: 0.61513156', 'Number: 3 - acc: 0.72492284  - val_acc: 0.61513156']
Optimizer: RMSprop lr: 0.01 Batch Size: 256
['Number: 0 - acc: 0.6097608  - val_acc: 0.7450658', 'Number: 2 - acc: 0.7172068  - val_acc: 0.61513156', 'Number: 3 - acc: 0.7355324  - val_acc: 0.7944079']
Optimizer: Adam lr: 0.01 Batch Size: 64
['Number: 0 - acc: 0.8097994  - val_acc: 0.68914473', 'Number: 2 - acc: 0.86381173  - val_acc: 0.83388156', 'Number: 3 - acc: 0.8458719  - val_acc: 0.8388158']
Optimizer: Adam lr: 0.01 Batch Size: 128
['Number: 0 - acc: 0.8165509  - val_acc: 0.8190789', 'Number: 2 - acc: 0.8553241  - val_acc: 0.8009868', 'Number: 3 - acc: 0.8159722  - val_acc: 0.7

Optimizer: Adam lr: 1e-05 Batch Size: 256
['Number: 0 - acc: 0.88348764  - val_acc: 0.8618421', 'Number: 2 - acc: 0.88753855  - val_acc: 0.8634868', 'Number: 3 - acc: 0.880787  - val_acc: 0.8618421']
Optimizer: RMSprop lr: 0.01 Batch Size: 64
['Number: 0 - acc: 0.7201003  - val_acc: 0.61513156', 'Number: 2 - acc: 0.74054784  - val_acc: 0.61513156', 'Number: 3 - acc: 0.69579476  - val_acc: 0.61513156']
Optimizer: RMSprop lr: 0.01 Batch Size: 128
['Number: 0 - acc: 0.6703318  - val_acc: 0.61513156', 'Number: 2 - acc: 0.7256944  - val_acc: 0.61513156', 'Number: 3 - acc: 0.71489197  - val_acc: 0.61513156']
Optimizer: RMSprop lr: 0.01 Batch Size: 256
['Number: 0 - acc: 0.677662  - val_acc: 0.61513156', 'Number: 2 - acc: 0.6533565  - val_acc: 0.61513156', 'Number: 3 - acc: 0.72723764  - val_acc: 0.61513156']
Optimizer: Adam lr: 0.01 Batch Size: 64
['Number: 0 - acc: 0.802662  - val_acc: 0.7384868', 'Number: 2 - acc: 0.8472222  - val_acc: 0.78453946', 'Number: 3 - acc: 0.8628472  - val_acc: 0

Optimizer: Adam lr: 1e-05 Batch Size: 128
['Number: 0 - acc: 0.9384645  - val_acc: 0.8618421', 'Number: 2 - acc: 0.9427083  - val_acc: 0.86019737', 'Number: 3 - acc: 0.943287  - val_acc: 0.8618421']
Optimizer: Adam lr: 1e-05 Batch Size: 256
['Number: 0 - acc: 0.9390432  - val_acc: 0.86513156', 'Number: 2 - acc: 0.94540894  - val_acc: 0.8618421', 'Number: 3 - acc: 0.94135803  - val_acc: 0.8618421']
Optimizer: RMSprop lr: 0.01 Batch Size: 64
['Number: 0 - acc: 0.6747685  - val_acc: 0.61513156', 'Number: 2 - acc: 0.732446  - val_acc: 0.61513156', 'Number: 3 - acc: 0.7361111  - val_acc: 0.61513156']
Optimizer: RMSprop lr: 0.01 Batch Size: 128
['Number: 0 - acc: 0.7025463  - val_acc: 0.61513156', 'Number: 2 - acc: 0.7141204  - val_acc: 0.61513156', 'Number: 3 - acc: 0.7083333  - val_acc: 0.61513156']
Optimizer: RMSprop lr: 0.01 Batch Size: 256
['Number: 0 - acc: 0.5243056  - val_acc: 0.61513156', 'Number: 2 - acc: 0.41666666  - val_acc: 0.3848684', 'Number: 3 - acc: 0.583912  - val_acc: 0.6

Optimizer: Adam lr: 1e-05 Batch Size: 64
['Number: 0 - acc: 0.93306327  - val_acc: 0.87171054', 'Number: 2 - acc: 0.9342207  - val_acc: 0.8700658', 'Number: 3 - acc: 0.9347994  - val_acc: 0.8503289']
Optimizer: Adam lr: 1e-05 Batch Size: 128
['Number: 0 - acc: 0.93364197  - val_acc: 0.8634868', 'Number: 2 - acc: 0.9322917  - val_acc: 0.8569079', 'Number: 3 - acc: 0.9349923  - val_acc: 0.86513156']
Optimizer: Adam lr: 1e-05 Batch Size: 256
['Number: 0 - acc: 0.9357639  - val_acc: 0.8684211', 'Number: 2 - acc: 0.9357639  - val_acc: 0.8684211', 'Number: 3 - acc: 0.9349923  - val_acc: 0.8684211']


In [10]:
best_params

{'batch_size': 128,
 'epochs': 11,
 'history': {'acc': [0.8879244,
   0.88522375,
   0.884838,
   0.88560957,
   0.8796296,
   0.88406634,
   0.8825231,
   0.880787,
   0.88503087,
   0.88233024,
   0.88560957,
   0.8825231,
   0.88001543,
   0.88059413,
   0.8784722,
   0.88522375,
   0.8813657,
   0.87808645,
   0.88445216,
   0.8825231],
  'loss': [0.42101160869186305,
   0.38889007583076574,
   0.38890349055513923,
   0.38696593119774336,
   0.38911284414338476,
   0.3899866320468761,
   0.38993777408276076,
   0.38866366501207705,
   0.38730849638397313,
   0.389200109022635,
   0.3849490737473523,
   0.3854871690273285,
   0.39020599075305606,
   0.3894362457004594,
   0.3915067999451249,
   0.387226680914561,
   0.387485996072675,
   0.39432238575852946,
   0.38422362230442186,
   0.39387339758284295],
  'val_acc': [0.8125,
   0.8305921,
   0.8684211,
   0.8865132,
   0.87335527,
   0.80263156,
   0.8256579,
   0.89638156,
   0.8059211,
   0.84210527,
   0.8996711,
   0.8125,
  

In [29]:
best_params

{'batch_size': 128,
 'epochs': 18,
 'lr': 0.0004,
 'optimizer': 'Adam',
 'val_acc': 0.8930921}

In [25]:
sorted_history = sorted(all_historys, key=lambda x:x['mean_val_acc'], reverse=True)

In [26]:
import pickle

In [27]:
with open('all_historys.pkl', 'wb') as out:
    pickle.dump(sorted_history, out)

In [29]:
with open('all_historys.pkl', 'rb') as in_history:
    sorted_history = pickle.load(in_history)

In [32]:
sorted_short_history = []
for item in sorted_history:
    history_line = item.copy()
    history_line.pop('history')
    sorted_short_history.append(history_line)

In [33]:
sorted_short_history

[{'batch_size': 64,
  'lr': 1e-05,
  'mean_val_acc': 0.8671052694320679,
  'optimizer': 'RMSprop'},
 {'batch_size': 128,
  'lr': 0.0001,
  'mean_val_acc': 0.8666118413209916,
  'optimizer': 'Adam'},
 {'batch_size': 128,
  'lr': 2e-05,
  'mean_val_acc': 0.8663651287555695,
  'optimizer': 'RMSprop'},
 {'batch_size': 256,
  'lr': 1e-05,
  'mean_val_acc': 0.8662828981876374,
  'optimizer': 'RMSprop'},
 {'batch_size': 128,
  'lr': 1e-05,
  'mean_val_acc': 0.8662828952074051,
  'optimizer': 'Adam'},
 {'batch_size': 64,
  'lr': 2e-05,
  'mean_val_acc': 0.8650493383407593,
  'optimizer': 'Adam'},
 {'batch_size': 256,
  'lr': 2e-05,
  'mean_val_acc': 0.8649671047925949,
  'optimizer': 'Adam'},
 {'batch_size': 128,
  'lr': 1e-05,
  'mean_val_acc': 0.8648026257753372,
  'optimizer': 'RMSprop'},
 {'batch_size': 256,
  'lr': 1e-05,
  'mean_val_acc': 0.8647203862667083,
  'optimizer': 'Adam'},
 {'batch_size': 64,
  'lr': 2e-05,
  'mean_val_acc': 0.864638152718544,
  'optimizer': 'RMSprop'},
 {'batch

In [37]:
for item in sorted_short_history:
    if item.get('val_acc'):
        print(item)

# KERAS TUNER

In [3]:
from kerastuner.tuners import RandomSearch


def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Range('units',
                                          min_value=128,
                                          max_value=384,
                                          step=64),
                           kernel_regularizer=tf.keras.regularizers.l1_l2(l2=0.01, l1=0.01),
                           activation='relu',
                           input_shape=(1024,)))
    model.add(tf.keras.layers.Dropout(hp.Choice('rate', values=[0.2, 0.35, 0.5])))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[0.0004, 0.0002, 0.0001])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [4]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='kerastunerlogs',
    project_name='chestXRay')

In [5]:
tuner.search_space_summary()

In [6]:
tuner.search(x=features_train,
             y=y_train,
             epochs=10,
             validation_data=(features_test, y_test),
            )

W0706 14:34:32.367661 140063488407296 deprecation.py:323] From /home/ivan/pybr/projeto/venv/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Name,Best model,Current model
accuracy,0.9095,0.9095
val_accuracy,0.8443,0.8443
val_loss,0.6363,0.6363
loss,0.5173,0.5173


Name,Best model,Current model
accuracy,0.9176,0.9176
val_accuracy,0.864,0.864
val_loss,0.5758,0.5758
loss,0.4244,0.4244


Name,Best model,Current model
accuracy,0.9224,0.9224
val_accuracy,0.864,0.8443
val_loss,0.5758,0.6335
loss,0.4244,0.4682


Name,Best model,Current model
accuracy,0.9224,0.9215
val_accuracy,0.864,0.8492
val_loss,0.5758,0.7005
loss,0.4244,0.5104


...

W0706 14:38:53.510124 140063488407296 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.118936). Check your callbacks.
W0706 14:38:53.523058 140063488407296 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.118936). Check your callbacks.
W0706 14:38:53.572865 140063488407296 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.118936). Check your callbacks.


Name,Best model,Current model
accuracy,0.9288,0.9255
val_accuracy,0.864,0.8558
val_loss,0.5758,1.4848
loss,0.4244,1.4673


W0706 14:39:19.847372 140063488407296 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.144399). Check your callbacks.
W0706 14:39:20.037859 140063488407296 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.172147). Check your callbacks.
W0706 14:39:20.153142 140063488407296 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.172147). Check your callbacks.
W0706 14:39:20.189420 140063488407296 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.172147). Check your callbacks.
W0706 14:39:20.217148 140063488407296 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.172147). Check your callbacks.
W0706 14:39:20.235585 140063488407296 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.151532). Check your callbacks.
W0706 14:39:20.247029 140063488407296 callbacks.py:241] Method (on_tra

Name,Best model,Current model
accuracy,0.9288,0.9208
val_accuracy,0.864,0.8251
val_loss,0.5758,1.6134
loss,0.4244,1.572


Name,Best model,Current model
accuracy,0.9288,0.9231
val_accuracy,0.864,0.8372
val_loss,0.5758,0.6113
loss,0.3934,0.3934


In [18]:
# Rodada 1

In [27]:
tuner.results_summary()

In [28]:
models = tuner.get_best_models(num_models=3)

In [29]:
models[0].summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


In [30]:
models[1].summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 192)               196800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 192)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 193       
Total params: 196,993
Trainable params: 196,993
Non-trainable params: 0
_________________________________________________________________


In [31]:
models[2].summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 192)               196800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 192)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 193       
Total params: 196,993
Trainable params: 196,993
Non-trainable params: 0
_________________________________________________________________


In [32]:
for n in range(3):
    print(tf.keras.backend.eval(models[n].optimizer.lr))

0.0002
0.0004
0.0004


In [ ]:
# Rodada 3

In [14]:
models = tuner.get_best_models(num_models=2)

In [19]:
tuner.results_summary()

In [15]:
models[0].summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               131200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 131,329
Trainable params: 131,329
Non-trainable params: 0
_________________________________________________________________


In [20]:
models[0].layers[1].rate

0.35

In [16]:
models[1].summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


In [17]:
models[1].layers[1].rate

0.5

In [8]:
for n in range(2):
    print(tf.keras.backend.eval(models[n].optimizer.lr))

0.0004
1e-04


In [7]:
models = tuner.get_best_models(num_models=2)

In [10]:
tuner.results_summary()

In [12]:
models[0].summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 320)               328000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 320)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 321       
Total params: 328,321
Trainable params: 328,321
Non-trainable params: 0
_________________________________________________________________


In [19]:
print(tf.keras.backend.eval(models[0].optimizer.lr))

0.0002


In [21]:
models[1].summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 320)               328000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 320)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 321       
Total params: 328,321
Trainable params: 328,321
Non-trainable params: 0
_________________________________________________________________


In [22]:
print(tf.keras.backend.eval(models[1].optimizer.lr))

1e-04


# Acima, foi utilizado tanto gridsearch quanto Random Search para tentar determinar o uso ideal de learningrate e outros hiperparâmetros. 

## Para finalizar a análise, será testado um treinamento maior e analisado também precision e recall dos modelos em quatro treinamentos: o primeiro com regularização E dropout, o segundo com dropout grande, o terceiro com dropout menor e o quarto sem dropout e sem regularização

### Todos utilizarão learning rate de 1e-4 com decay

In [3]:
from sklearn.metrics import classification_report, confusion_matrix

def test_model(model):
    y_pred = model.predict(features_test) > 0.5
    print('METRICS ON TEST SET')
    print(classification_report(y_test, y_pred, target_names=['NORMAL', 'PNEUMONIA']))
    print(confusion_matrix(y_test, y_pred))
    y_pred_train = model.predict(features_train) > 0.5
    print('METRICS ON TRAIN SET')
    print(classification_report(y_train, y_pred_train, target_names=['NORMAL', 'PNEUMONIA']))
    print(confusion_matrix(y_train, y_pred_train))

In [4]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.8, patience=3,
                              verbose=1, mode='auto')


In [5]:
models = []
historys = []
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=256,
                       kernel_regularizer=tf.keras.regularizers.l1_l2(l2=0.01, l1=0.01),
                       activation='relu',
                       input_shape=(1024,)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0004),
    loss='binary_crossentropy',
    metrics=['accuracy'])
models.append(model)

for dropout_rate in [0.4, 0.2, 0]:
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=256,
                           activation='relu',
                           input_shape=(1024,)))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0004),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    models.append(model)

In [6]:
# Model0: regularização e Dropout
historys.append(models[0].fit(x=features_train,
                    y=y_train,
                    batch_size=256,
                    epochs=100,
                    verbose=2,
                    callbacks=[early_stop, reduce_lr],
                    validation_data=(features_test, y_test),
                   )
               )

W0707 15:49:04.480043 139748519900928 deprecation.py:323] From /home/ivan/pybr/projeto/venv/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 5184 samples, validate on 608 samples
Epoch 1/100
5184/5184 - 0s - loss: 87.4591 - accuracy: 0.7145 - val_loss: 78.2496 - val_accuracy: 0.6826
Epoch 2/100
5184/5184 - 0s - loss: 70.4839 - accuracy: 0.8519 - val_loss: 62.2767 - val_accuracy: 0.8306
Epoch 3/100
5184/5184 - 0s - loss: 55.7412 - accuracy: 0.8802 - val_loss: 48.8534 - val_accuracy: 0.7664
Epoch 4/100
5184/5184 - 0s - loss: 42.9798 - accuracy: 0.8978 - val_loss: 37.0173 - val_accuracy: 0.8207
Epoch 5/100
5184/5184 - 0s - loss: 32.2591 - accuracy: 0.9041 - val_loss: 27.3722 - val_accuracy: 0.8536
Epoch 6/100
5184/5184 - 0s - loss: 23.6079 - accuracy: 0.9128 - val_loss: 19.9115 - val_accuracy: 0.8174
Epoch 7/100
5184/5184 - 0s - loss: 17.0166 - accuracy: 0.9225 - val_loss: 14.5062 - val_accuracy: 0.7763
Epoch 8/100

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.00031999999191612005.
5184/5184 - 0s - loss: 12.4402 - accuracy: 0.9252 - val_loss: 11.0077 - val_accuracy: 0.7862
Epoch 9/100
5184/5184 - 0s - lo

Epoch 65/100
5184/5184 - 0s - loss: 1.0541 - accuracy: 0.9325 - val_loss: 1.2607 - val_accuracy: 0.8076
Epoch 66/100

Epoch 00066: ReduceLROnPlateau reducing learning rate to 4.6116860175970945e-06.
5184/5184 - 0s - loss: 1.0504 - accuracy: 0.9311 - val_loss: 1.2556 - val_accuracy: 0.8076
Epoch 67/100
5184/5184 - 0s - loss: 1.0447 - accuracy: 0.9325 - val_loss: 1.2547 - val_accuracy: 0.8076
Epoch 68/100
5184/5184 - 0s - loss: 1.0444 - accuracy: 0.9321 - val_loss: 1.2494 - val_accuracy: 0.8076
Epoch 69/100

Epoch 00069: ReduceLROnPlateau reducing learning rate to 3.6893488868372515e-06.
5184/5184 - 0s - loss: 1.0436 - accuracy: 0.9300 - val_loss: 1.2486 - val_accuracy: 0.8076
Epoch 70/100
5184/5184 - 0s - loss: 1.0392 - accuracy: 0.9306 - val_loss: 1.2443 - val_accuracy: 0.8076
Epoch 71/100
5184/5184 - 0s - loss: 1.0353 - accuracy: 0.9336 - val_loss: 1.2427 - val_accuracy: 0.8076
Epoch 72/100

Epoch 00072: ReduceLROnPlateau reducing learning rate to 2.9514791094698013e-06.
5184/5184 - 0

In [7]:
# Model1: Dropout alto
historys.append(models[1].fit(x=features_train,
                    y=y_train,
                    batch_size=256,
                    epochs=100,
                    verbose=2,
                    callbacks=[early_stop, reduce_lr],
                    validation_data=(features_test, y_test),
                   )
               )

Train on 5184 samples, validate on 608 samples
Epoch 1/100
5184/5184 - 0s - loss: 0.8020 - accuracy: 0.7496 - val_loss: 0.7672 - val_accuracy: 0.7319
Epoch 2/100
5184/5184 - 0s - loss: 0.3319 - accuracy: 0.8779 - val_loss: 0.4452 - val_accuracy: 0.8339
Epoch 3/100
5184/5184 - 0s - loss: 0.2475 - accuracy: 0.8968 - val_loss: 0.4381 - val_accuracy: 0.8240
Epoch 4/100
5184/5184 - 0s - loss: 0.2043 - accuracy: 0.9176 - val_loss: 0.5296 - val_accuracy: 0.7878
Epoch 5/100
5184/5184 - 0s - loss: 0.1962 - accuracy: 0.9205 - val_loss: 0.3731 - val_accuracy: 0.8470
Epoch 6/100
5184/5184 - 0s - loss: 0.1740 - accuracy: 0.9323 - val_loss: 0.4162 - val_accuracy: 0.8306
Epoch 7/100
5184/5184 - 0s - loss: 0.1704 - accuracy: 0.9321 - val_loss: 0.3886 - val_accuracy: 0.8421
Epoch 8/100
5184/5184 - 0s - loss: 0.1484 - accuracy: 0.9396 - val_loss: 0.3888 - val_accuracy: 0.8487
Epoch 9/100
5184/5184 - 0s - loss: 0.1488 - accuracy: 0.9414 - val_loss: 0.3827 - val_accuracy: 0.8454
Epoch 10/100
5184/5184 - 0

In [8]:
# Model2: Dropout baixo
historys.append(models[2].fit(x=features_train,
                    y=y_train,
                    batch_size=256,
                    epochs=100,
                    verbose=2,
                    callbacks=[early_stop, reduce_lr],
                    validation_data=(features_test, y_test),
                   )
               )

Train on 5184 samples, validate on 608 samples
Epoch 1/100
5184/5184 - 0s - loss: 0.4232 - accuracy: 0.8439 - val_loss: 0.6293 - val_accuracy: 0.7697
Epoch 2/100
5184/5184 - 0s - loss: 0.2296 - accuracy: 0.9151 - val_loss: 0.4920 - val_accuracy: 0.8043
Epoch 3/100
5184/5184 - 0s - loss: 0.1780 - accuracy: 0.9300 - val_loss: 0.4336 - val_accuracy: 0.8174
Epoch 4/100
5184/5184 - 0s - loss: 0.1597 - accuracy: 0.9360 - val_loss: 0.3665 - val_accuracy: 0.8586
Epoch 5/100
5184/5184 - 0s - loss: 0.1492 - accuracy: 0.9387 - val_loss: 0.4547 - val_accuracy: 0.8207
Epoch 6/100
5184/5184 - 0s - loss: 0.1457 - accuracy: 0.9400 - val_loss: 0.3617 - val_accuracy: 0.8635
Epoch 7/100
5184/5184 - 0s - loss: 0.1258 - accuracy: 0.9512 - val_loss: 0.3669 - val_accuracy: 0.8520
Epoch 8/100
5184/5184 - 0s - loss: 0.1143 - accuracy: 0.9578 - val_loss: 0.3699 - val_accuracy: 0.8536
Epoch 9/100

Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.00031999999191612005.
5184/5184 - 0s - loss: 0.1106 - acc

In [9]:
# Model3: Sem Dropout 
historys.append(models[3].fit(x=features_train,
                    y=y_train,
                    batch_size=256,
                    epochs=100,
                    verbose=2,
                    callbacks=[early_stop, reduce_lr],
                    validation_data=(features_test, y_test),
                   )
               )

Train on 5184 samples, validate on 608 samples
Epoch 1/100
5184/5184 - 0s - loss: 0.4907 - accuracy: 0.8084 - val_loss: 0.6838 - val_accuracy: 0.7336
Epoch 2/100
5184/5184 - 0s - loss: 0.2374 - accuracy: 0.8989 - val_loss: 0.6404 - val_accuracy: 0.7632
Epoch 3/100
5184/5184 - 0s - loss: 0.1892 - accuracy: 0.9242 - val_loss: 0.4896 - val_accuracy: 0.8026
Epoch 4/100
5184/5184 - 0s - loss: 0.1724 - accuracy: 0.9302 - val_loss: 0.5157 - val_accuracy: 0.7977
Epoch 5/100
5184/5184 - 0s - loss: 0.1497 - accuracy: 0.9435 - val_loss: 0.3961 - val_accuracy: 0.8421
Epoch 6/100
5184/5184 - 0s - loss: 0.1362 - accuracy: 0.9497 - val_loss: 0.4683 - val_accuracy: 0.8092
Epoch 7/100
5184/5184 - 0s - loss: 0.1301 - accuracy: 0.9508 - val_loss: 0.3175 - val_accuracy: 0.8832
Epoch 8/100
5184/5184 - 0s - loss: 0.1240 - accuracy: 0.9514 - val_loss: 0.4060 - val_accuracy: 0.8470
Epoch 9/100
5184/5184 - 0s - loss: 0.1159 - accuracy: 0.9581 - val_loss: 0.3152 - val_accuracy: 0.8799
Epoch 10/100

Epoch 00010:

In [10]:
test_model(models[0])

METRICS ON TEST SET
              precision    recall  f1-score   support

      NORMAL       0.89      0.57      0.69       234
   PNEUMONIA       0.78      0.96      0.86       374

    accuracy                           0.81       608
   macro avg       0.84      0.76      0.78       608
weighted avg       0.82      0.81      0.80       608

[[133 101]
 [ 16 358]]
METRICS ON TRAIN SET
              precision    recall  f1-score   support

      NORMAL       0.90      0.86      0.88      1341
   PNEUMONIA       0.95      0.97      0.96      3843

    accuracy                           0.94      5184
   macro avg       0.92      0.91      0.92      5184
weighted avg       0.94      0.94      0.94      5184

[[1156  185]
 [ 134 3709]]


In [11]:
test_model(models[1])

METRICS ON TEST SET
              precision    recall  f1-score   support

      NORMAL       0.95      0.67      0.79       234
   PNEUMONIA       0.83      0.98      0.90       374

    accuracy                           0.86       608
   macro avg       0.89      0.82      0.84       608
weighted avg       0.87      0.86      0.85       608

[[157  77]
 [  8 366]]
METRICS ON TRAIN SET
              precision    recall  f1-score   support

      NORMAL       0.96      0.95      0.95      1341
   PNEUMONIA       0.98      0.99      0.98      3843

    accuracy                           0.98      5184
   macro avg       0.97      0.97      0.97      5184
weighted avg       0.98      0.98      0.98      5184

[[1271   70]
 [  57 3786]]


In [12]:
test_model(models[2])

METRICS ON TEST SET
              precision    recall  f1-score   support

      NORMAL       0.93      0.71      0.81       234
   PNEUMONIA       0.84      0.97      0.90       374

    accuracy                           0.87       608
   macro avg       0.89      0.84      0.86       608
weighted avg       0.88      0.87      0.87       608

[[167  67]
 [ 12 362]]
METRICS ON TRAIN SET
              precision    recall  f1-score   support

      NORMAL       0.96      0.97      0.96      1341
   PNEUMONIA       0.99      0.99      0.99      3843

    accuracy                           0.98      5184
   macro avg       0.97      0.98      0.98      5184
weighted avg       0.98      0.98      0.98      5184

[[1297   44]
 [  54 3789]]


In [13]:
test_model(models[3])

METRICS ON TEST SET
              precision    recall  f1-score   support

      NORMAL       0.97      0.56      0.71       234
   PNEUMONIA       0.78      0.99      0.87       374

    accuracy                           0.83       608
   macro avg       0.88      0.78      0.79       608
weighted avg       0.86      0.83      0.81       608

[[132 102]
 [  4 370]]
METRICS ON TRAIN SET
              precision    recall  f1-score   support

      NORMAL       0.97      0.89      0.93      1341
   PNEUMONIA       0.96      0.99      0.98      3843

    accuracy                           0.96      5184
   macro avg       0.97      0.94      0.95      5184
weighted avg       0.96      0.96      0.96      5184

[[1190  151]
 [  33 3810]]


# CONCLUSÕES

## Aparentemente, a learningrate ideal ficou em torno de 0.0004 a 0.0001
## A regularização prejudicou o aprendizado sem causar grande impacto na generalização
## Em primeiro lugar, é preciso buscar um bom resultado no treinamento, para em seguida tentar melhorar a generalização
## A utilização de dropout também não causou grande ganho na generalização, impactando negativamente (embora pouco)  o aprendizado
## Houve ganho significativo em relação à rede neural simples: (acc: 0,97  - val_acc: 0,88 X acc: 0,95 - val_acc: 0,85) mesmo sem tunning das camadas convolucionais
## O desempenho é bem pior na classe NORMAL, provavelmente devido ao dataset estar desbalanceado



# PRÓXIMOS PASSOS

## Utilizar os pesos aprendidos no melhor modelo models[2] para treinar TransferLearning do zero (sem feature extraction) e fazer tunning de algumas camadas convolucionais, utilizando extensamente ImageAugmentation para evitar sobreajuste e compensar desbalancemanto. Utilizar recall de pneumonia como métrica adicional, por ser o mais importante neste caso (tentar algo próximo de 100%)




In [14]:
models[2].save_weights('../modelschest/02d.h5')